In [ ]:
import sys,os
sys.path.insert(1,os.getcwd())
import numpy as np
import utils as ut
from batscode import bats_box
import matplotlib.pyplot as plt

In [ ]:
bats = bats_box()
ac_list = []
tg_list = []
graph_list = []
dpkts = []
runs = 1000
for i in range(runs):
    t1 = ut.gen_tanner_all_connected(n_pkt=64,n_batch=9,bsize=8,pk_size=256,silent=True,deg_dist='./K64M8.txt')
    bats.update_graph(t1)
    ac, tg, dpkt = bats.run_avg(num_hops=10,loss_rate=0.1,inactivation=True,silent=True,return_dpkt=True)
    # (ac, tg, _), dpkt = bats.run(num_hops=10,loss_rate=0.1,inactivation=True,silent=True)
    ac_list.append(ac/tg)
    tg_list.append(bats.pkt_per_edge_)
    graph_list.append(t1)
    dpkts += dpkt
    print('\r%d/%d'%(i+1,runs),end='')
print('\n')
print(np.mean(ac_list))
print(np.mean(tg_list))
print(np.std(ac_list))
print(np.std(tg_list))

In [ ]:
bats = bats_box()
# t1 = ut.gen_tanner(n_pkt=64,n_batch=11,bsize=8,pk_size=256,silent=False,deg_dist='./K64M8.txt')
t1 = ut.gen_tanner_all_connected(n_pkt=64,n_batch=12,bsize=8,pk_size=256,silent=False,deg_dist='./K64M8.txt')
bats.update_graph(t1)
t1.info()
ac, tg, dpkt = bats.run_avg(num_hops=20,loss_rate=0.1,inactivation=True,silent=True,return_dpkt=True)
print(ac/tg)
print(bats.pkt_per_edge_)

In [ ]:
# basic decoable graph
def gen_basis_graph_from_model():
    n_cns = 10
    n_vns = 64
    arr_to_return = np.zeros((n_cns,n_vns),dtype=int)
    available_cns = list(np.arange(n_cns,dtype=int))
    for i in range(n_vns):
        if len(available_cns) == 0:
            available_cns = list(np.arange(n_cns,dtype=int))
        idx = np.random.choice(available_cns,1)[0]
        available_cns.remove(idx)
        arr_to_return[idx,i] = 1
    return arr_to_return

bats = bats_box()
ac_list = []
tg_list = []
graph_list = []
for i in range(100):
    current_state_ = gen_basis_graph_from_model()
    t1 = ut.gen_tanner_from_binary_graph(current_state_, 256, 8)
    bats.update_graph(t1)
    ac, tg = bats.run_avg(num_hops=20,loss_rate=0.1,inactivation=True,silent=True)
    ac_list.append(ac/tg)
    tg_list.append(bats.pkt_per_edge_)
    graph_list.append(t1)
    print('\r%d/%d'%(i+1,100),end='')
print('\n')
print(np.mean(ac_list))
print(np.mean(tg_list))
print(np.std(ac_list))
print(np.std(tg_list))

In [ ]:
# get rank distribution
from scipy.io import savemat

def drop(bsize,loss_rate):
    idx = []
    for i in range(bsize):
        if np.random.rand() < loss_rate:
            idx.append(i)
    return idx
def rank_test(num_hops, loss_rate, M):
    b = ut.gf(np.random.randint(256,size=(256,M)))
    while np.linalg.matrix_rank(b) != M:
            b = ut.gf(np.random.randint(256,size=(256,M)))
    for _ in range(num_hops):
        h = ut.gf(np.random.randint(256,size=(M,M)))
        while np.linalg.matrix_rank(h) != M:
            h = ut.gf(np.random.randint(256,size=(M,M)))
        drop_idx = drop(M,loss_rate)
        keep_index = [i for i in range(M) if i not in drop_idx]
        if len(keep_index) == 0:
            return 0
        b = np.matmul(b[:,keep_index], h[keep_index,:])
    return np.linalg.matrix_rank(b)

def get_rank_dist(num_hops, loss_rate, M):
    num_tests = int(1e4)
    dist = np.zeros(M+1)
    for i in range(num_tests):
        print('\r%d/%d'%(i+1,num_tests),end='')
        dist[int(rank_test(num_hops, loss_rate, M))] += 1
    dist = dist / num_tests
    return dist


In [ ]:
num_hops = 10
loss_rate = 0.1
M = 8
dist = get_rank_dist(num_hops,loss_rate, M)

In [ ]:
matfile = {
    'h': dist
}
savemat('rank_dist_M%d.mat'%M, matfile)

In [ ]:
dist